In [ ]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.2/320.2 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 84.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.2/168.2 kB 10.3 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
import json
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments, TextDataset, DataCollatorForLanguageModeling

# Paths to your data
data_path = "/content/mentalhealth_llm.json"  # Replace with your LLM-compatible file path
model_output_dir = "./mental_health_llm_model"

# Load your dataset
def prepare_text_dataset(data_path):
    with open(data_path, 'r') as f:
        data = json.load(f)

    dataset = []
    for item in data:
        instruction = item['instruction']
        response = item['response']
        # Combine instruction and response in a conversational format
        conversation = f"Instruction: {instruction}\nResponse: {response}\n"
        dataset.append(conversation)

    return "\n".join(dataset)

# Save processed dataset to text file (GPT-2 expects plain text format)
prepared_data = prepare_text_dataset(data_path)
text_dataset_path = "prepared_data.txt"
with open(text_dataset_path, "w") as f:
    f.write(prepared_data)

# Load GPT-2 tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Prepare dataset for training
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=text_dataset_path,
    block_size=128  # Length of each training sequence
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Set to False for causal language modeling
)

# Define training arguments
training_args = TrainingArguments(
    output_dir=model_output_dir,
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator
)

# Train the model
trainer.train()

# Save the fine-tuned model and tokenizer
model.save_pretrained(model_output_dir)
tokenizer.save_pretrained(model_output_dir)

print(f"Model saved to {model_output_dir}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss


Model saved to ./mental_health_llm_model


In [ ]:
def prepare_text_dataset(data_path):
    with open(data_path, 'r') as f:
        data = json.load(f)

    dataset = []
    for item in data:
        instruction = item['instruction']
        response = item['response']
        # Create plain conversation text
        conversation = f"{instruction}\n{response}\n"
        dataset.append(conversation)

    return "\n".join(dataset)

In [ ]:
def chat_with_model(input_text):
    # Tokenize the input
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    # Generate response
    output = model.generate(
        input_ids,
        max_length=150,  # Adjust as needed for concise responses
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.2,
        top_k=50
    )
    # Decode the response and strip out any instructions
    decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
    # Ensure only the relevant response is returned
    return decoded_output.strip()


In [ ]:
#imp
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the fine-tuned model and tokenizer
model_path = "./mental_health_llm_model"  # Replace with your output directory
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)

def clip_after_second_instruction(input_string):
    """
    Clips the string after the second occurrence of the word 'Instruction'.

    Parameters:
        input_string (str): The input string to be processed.

    Returns:
        str: The clipped string if 'Instruction' appears twice, otherwise the original string.
    """
    keyword = "Instruction"
    first_occurrence = input_string.find(keyword)

    if first_occurrence == -1:
        # 'Instruction' not found
        return input_string

    # Find the second occurrence
    second_occurrence = input_string.find(keyword, first_occurrence + len(keyword))

    if second_occurrence != -1:
        # Clip the string at the second occurrence
        return input_string[:second_occurrence]
    else:
        # Return the original string if 'Instruction' occurs only once
        return input_string


# Generate response
def chat_with_model(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    output = model.generate(input_ids, max_length=100, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

# Test the LLM
while True:
    user_input = input("You: ")
    if user_input.lower() in ["quit", "exit"]:
        break
    response = chat_with_model(f"Instruction: {user_input}\nResponse:")
    # response = chat_with_model(user_input)
    print(f"LLM: {clip_after_second_instruction(response)}")


You: hi
LLM: Instruction: hi
Response: Hi there. How are you feeling today?


You: feeling good thanks for asking
LLM: Instruction: feeling good thanks for asking
Response: I'm glad you're feeling this way. I'm listening.


You: exit


This is an Instruction to follow. Another 


In [ ]:
import gradio as gr

# Define a Gradio chat interface
def chat_interface(user_input):
    return generate_response(user_input)

gr.Interface(fn=chat_interface, inputs="text", outputs="text", title="Mental Health LLM").launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://16c113916af519f043.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
